In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

In [2]:
def cnn_model_fn(features, labels, mode):
  
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  # Convolutional Layer #1
  # Computes 32 features using a 5x5 filter with ReLU activation.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 28, 28, 1]
  # Output Tensor Shape: [batch_size, 28, 28, 32]
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  # First max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 28, 28, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 32]
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2
  # Computes 64 features using a 5x5 filter.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 14, 14, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 64]
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #2
  # Second max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 14, 14, 64]
  # Output Tensor Shape: [batch_size, 7, 7, 64]
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Flatten tensor into a batch of vectors
  # Input Tensor Shape: [batch_size, 7, 7, 64]
  # Output Tensor Shape: [batch_size, 7 * 7 * 64]
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])

  # Dense Layer
  # Densely connected layer with 1024 neurons
  # Input Tensor Shape: [batch_size, 7 * 7 * 64]
  # Output Tensor Shape: [batch_size, 1024]
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)

  # Add dropout operation; 0.6 probability that element will be kept
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits layer
  # Input Tensor Shape: [batch_size, 1024]
  # Output Tensor Shape: [batch_size, 10]
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [3]:
def main(unused_argv):
  # Load training and eval data
  mnist = tf.contrib.learn.datasets.load_dataset("mnist")
  train_data = mnist.train.images  # Returns np.array
  train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
  eval_data = mnist.test.images  # Returns np.array
  eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

  # Create the Estimator
  mnist_classifier = tf.estimator.Estimator(
      model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model_102")

  # Set up logging for predictions
  # Log the values in the "Softmax" tensor with label "probabilities"
  tensors_to_log = {"probabilities": "softmax_tensor"}
  logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=50)

  # Train the model
  train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=50,
      num_epochs=None,
      shuffle=True)
  mnist_classifier.train(
      input_fn=train_input_fn,
      steps=3000,
      hooks=[logging_hook])

  # Evaluate the model and print results
  eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": eval_data},
      y=eval_labels,
      num_epochs=1,
      shuffle=False)
  eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
  print(eval_results)


if __name__ == "__main__":
  tf.app.run()

Extracting MNIST-data\train-images-idx3-ubyte.gz
Extracting MNIST-data\train-labels-idx1-ubyte.gz
Extracting MNIST-data\t10k-images-idx3-ubyte.gz
Extracting MNIST-data\t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model_102', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001F5198A7240>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INF

INFO:tensorflow:global_step/sec: 6.55178
INFO:tensorflow:probabilities = [[0.0003019  0.00000131 0.00014716 0.00006319 0.00036695 0.00058922
  0.00237259 0.00149969 0.9934314  0.00122663]
 [0.00005446 0.00034382 0.00000813 0.00212659 0.07777037 0.00059129
  0.00000228 0.06724703 0.00867311 0.843183  ]
 [0.00001382 0.         0.00000062 0.00000005 0.0000001  0.00001446
  0.00000004 0.00000181 0.9999386  0.00003057]
 [0.0000004  0.00001424 0.0000003  0.9854174  0.00012271 0.01366596
  0.00000579 0.00008174 0.00068242 0.00000903]
 [0.00000352 0.00034319 0.00000168 0.00025897 0.98721004 0.00025196
  0.00000652 0.00772342 0.00052918 0.00367156]
 [0.00004463 0.00042047 0.00018427 0.00327324 0.00011462 0.00194086
  0.00004979 0.0000767  0.9938539  0.00004142]
 [0.9998355  0.         0.00000786 0.00000047 0.00000242 0.00002642
  0.00005678 0.00000527 0.00006322 0.00000218]
 [0.00002923 0.99674916 0.00004058 0.00039504 0.00035475 0.00007098
  0.00040345 0.00101408 0.00082822 0.0001145 ]
 [0.009

INFO:tensorflow:global_step/sec: 6.67325
INFO:tensorflow:probabilities = [[0.         0.         0.9999999  0.00000006 0.         0.
  0.         0.00000004 0.         0.        ]
 [0.00000178 0.00000001 0.00000019 0.00140633 0.00000151 0.9979983
  0.00000007 0.0000113  0.00006507 0.0005154 ]
 [0.9317553  0.00001623 0.00089528 0.00040938 0.00001037 0.0003185
  0.05853892 0.00000348 0.00004826 0.00800429]
 [0.00001737 0.00000067 0.00002286 0.00001883 0.9817405  0.00000065
  0.00000586 0.00006257 0.00000302 0.01812763]
 [0.00053434 0.00005167 0.00006864 0.00000555 0.00006418 0.00028698
  0.9988477  0.00000128 0.00012161 0.00001798]
 [0.99999845 0.         0.         0.00000001 0.         0.00000014
  0.00000009 0.         0.00000003 0.00000137]
 [0.00000006 0.00022972 0.01472337 0.9850293  0.         0.00000261
  0.         0.00000005 0.00000399 0.00001089]
 [0.00002789 0.9998909  0.00001414 0.00000912 0.00001377 0.0000009
  0.0000015  0.0000032  0.00000558 0.00003282]
 [0.00004116 0.000

INFO:tensorflow:global_step/sec: 6.79746
INFO:tensorflow:probabilities = [[0.00000069 0.00007802 0.00000018 0.00007205 0.012933   0.00014093
  0.00000019 0.00049344 0.00018055 0.986101  ]
 [0.00000073 0.00000164 0.00290637 0.0007431  0.00000052 0.
  0.         0.9961416  0.00000728 0.00019877]
 [0.00000047 0.00000039 0.00000052 0.00017793 0.00203254 0.00000916
  0.0000003  0.00001415 0.00010648 0.997658  ]
 [0.00003806 0.00010301 0.93469405 0.00060606 0.00000738 0.00000021
  0.00000001 0.0374737  0.00042821 0.02664932]
 [0.99998415 0.         0.00000295 0.00000003 0.00000006 0.00000021
  0.00000354 0.00000056 0.00000005 0.00000841]
 [0.         0.00000502 0.00000173 0.9999565  0.00000075 0.00000822
  0.         0.00000283 0.0000052  0.00001967]
 [0.00001973 0.99973184 0.00000097 0.00000003 0.00011983 0.0000002
  0.00007091 0.00000083 0.00005312 0.00000249]
 [0.00000245 0.00000018 0.00017099 0.9992042  0.00000001 0.00014177
  0.00000003 0.00000124 0.00046693 0.00001226]
 [0.00001024 0.0

INFO:tensorflow:global_step/sec: 6.77477
INFO:tensorflow:probabilities = [[0.         0.0000013  0.00000093 0.999042   0.00000348 0.00087346
  0.00000001 0.00000022 0.00007849 0.00000016]
 [0.00007334 0.00010817 0.99945754 0.00000209 0.00000032 0.00000089
  0.0000001  0.00000851 0.0003345  0.00001459]
 [0.00000005 0.00005026 0.         0.         0.99994624 0.
  0.00000215 0.00000128 0.00000001 0.        ]
 [0.00000189 0.00000593 0.00167922 0.00000059 0.00000177 0.00017778
  0.0000936  0.00000002 0.99803716 0.00000196]
 [0.00000001 0.00001275 0.00003896 0.9998135  0.00004927 0.00003847
  0.00000115 0.00001286 0.00002974 0.00000329]
 [0.00000005 0.00000315 0.         0.         0.00000006 0.00000023
  0.99999654 0.         0.         0.        ]
 [0.00000012 0.99999106 0.0000003  0.00000001 0.0000021  0.00000002
  0.00000621 0.00000002 0.00000003 0.00000007]
 [0.00001594 0.00000393 0.9972977  0.00251411 0.00000004 0.00000029
  0.00000424 0.00015385 0.00000972 0.00000023]
 [0.         0.

...

INFO:tensorflow:global_step/sec: 6.88502
INFO:tensorflow:probabilities = [[0.         0.         0.         0.         0.00001303 0.
  0.         0.         0.00000001 0.999987  ]
 [0.         0.         0.00000001 0.99996686 0.         0.00000009
  0.         0.0000035  0.00000004 0.0000296 ]
 [0.         0.         0.         0.         0.         0.
  0.         0.99999857 0.         0.00000148]
 [0.00000001 0.00001715 0.00055742 0.00000057 0.00029872 0.00000003
  0.00000004 0.00000066 0.9991253  0.00000015]
 [0.00000001 0.         0.00000001 0.00000001 0.         0.00000196
  0.99999464 0.         0.00000334 0.        ]
 [0.         0.0000004  0.99999964 0.00000002 0.         0.
  0.         0.         0.00000003 0.        ]
 [0.9999981  0.         0.00000001 0.         0.00000004 0.
  0.00000178 0.         0.00000008 0.00000001]
 [0.         0.         0.         1.         0.         0.
  0.         0.         0.         0.        ]
 [0.0000252  0.00000037 0.00493686 0.30511478 0

INFO:tensorflow:global_step/sec: 6.92135
INFO:tensorflow:probabilities = [[0.00000273 0.00100408 0.00001972 0.15306917 0.00029156 0.0330578
  0.00000057 0.00000171 0.00190083 0.81065184]
 [0.         0.00000003 0.00000001 0.         0.00000001 0.00000001
  0.         0.99999976 0.         0.00000006]
 [0.99999905 0.         0.00000024 0.00000002 0.         0.
  0.00000009 0.0000001  0.0000001  0.00000041]
 [0.00000001 0.00114338 0.00000702 0.00000006 0.00031303 0.01271419
  0.002745   0.00000011 0.9830767  0.00000062]
 [0.         0.         0.         0.         0.99999964 0.
  0.         0.         0.         0.00000034]
 [0.9999976  0.         0.00000148 0.00000001 0.         0.00000008
  0.00000037 0.00000004 0.00000022 0.00000022]
 [0.         0.00000014 0.00000026 0.00000001 0.         0.
  0.         0.9999995  0.         0.00000009]
 [0.00000001 0.00002399 0.00001147 0.00000653 0.00000003 0.00000002
  0.         0.99994564 0.00000007 0.00001216]
 [0.00000054 0.00000033 0.999182

INFO:tensorflow:Saving checkpoints for 3000 into /tmp/mnist_convnet_model_102\model.ckpt.
INFO:tensorflow:Loss for final step: 0.016644517.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-16-16:36:25
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model_102\model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-04-16-16:36:34
INFO:tensorflow:Saving dict for global step 3000: accuracy = 0.9885, global_step = 3000, loss = 0.038777888
{'accuracy': 0.9885, 'loss': 0.038777888, 'global_step': 3000}


SystemExit: 

c:\users\ninad\anaconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
